## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-30-13-00-00 +0000,nypost,"AI influencers are now boasting personalities,...",https://nypost.com/2025/11/30/world-news/ai-in...
1,2025-11-30-12-52-32 +0000,bbc,Four killed in shooting at family gathering in...,https://www.bbc.com/news/articles/cvg81p61dnmo...
2,2025-11-30-12-42-44 +0000,nyt,Netanyahu Asks Israel’s President to Pardon Hi...,https://www.nytimes.com/2025/11/30/world/middl...
3,2025-11-30-12-31-39 +0000,nyt,Hong Kong’s Migrant Domestic Workers Mourn The...,https://www.nytimes.com/2025/11/30/world/asia/...
4,2025-11-30-12-23-44 +0000,bbc,Kazakh pair accused of using spycam and earpie...,https://www.bbc.com/news/articles/c075zd0j2j3o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
200,trump,21
66,up,8
306,dead,8
33,fire,8
357,russia,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
127,2025-11-29-20-06-00 +0000,wsj,Trump Declares That Airspace Around Venezuela ...,https://www.wsj.com/politics/national-security...,47
97,2025-11-29-22-53-36 +0000,bbc,Trump to pardon ex-Honduras president convicte...,https://www.bbc.com/news/articles/c1kpm0rvxepo...,46
139,2025-11-29-19-03-00 +0000,wsj,Why Russia and China Are Sitting Out Venezuela...,https://www.wsj.com/world/americas/why-russia-...,44
120,2025-11-29-20-49-01 +0000,cbc,Maduro slams 'colonial threat' as Trump says V...,https://www.cbc.ca/news/world/trump-maduro-ven...,44
88,2025-11-30-00-16-05 +0000,nyt,The Ex-President Whom Trump Plans to Pardon Fl...,https://www.nytimes.com/2025/11/29/nyregion/ho...,43


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
127,47,2025-11-29-20-06-00 +0000,wsj,Trump Declares That Airspace Around Venezuela ...,https://www.wsj.com/politics/national-security...
6,39,2025-11-30-12-13-41 +0000,bbc,Hong Kong fire death toll rises to 146 as thou...,https://www.bbc.com/news/articles/cp34yk7z6yxo...
13,32,2025-11-30-11-15-39 +0000,nypost,Netanyahu submits request to Israeli president...,https://nypost.com/2025/11/30/world-news/netan...
170,31,2025-11-29-13-50-50 +0000,nypost,Russia attack on Kyiv kills three as efforts t...,https://nypost.com/2025/11/29/world-news/russi...
82,31,2025-11-30-01-08-54 +0000,nypost,"Heartless son allegedly beat his parents, set ...",https://nypost.com/2025/11/29/us-news/lunatic-...
66,31,2025-11-30-03-50-26 +0000,nypost,"New York, New Jersey could get up to 5 inches ...",https://nypost.com/2025/11/29/us-news/ny-nj-co...
129,31,2025-11-29-20-03-34 +0000,nypost,Pope Leo visits Turkey’s famous Blue Mosque in...,https://nypost.com/2025/11/29/world-news/pope-...
7,25,2025-11-30-12-00-28 +0000,nyt,"Tom Stoppard, Award-Winning Playwright of Witt...",https://www.nytimes.com/2025/11/29/theater/tom...
32,24,2025-11-30-10-00-00 +0000,wapo,Why you may not want lower prices as much as y...,https://www.washingtonpost.com/business/2025/1...
42,23,2025-11-30-09-00-00 +0000,cbc,A 'more serious public image': How the Princes...,https://www.cbc.ca/news/world/catherine-prince...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
